In [1]:
import pandas as pd
import gc
gc.collect()
import datetime
import hashlib
import numpy as np
import logging

In [ ]:
logging.basicConfig(filename='merge_sales_with_signing_up', level=logging.INFO)
logging.info("Start Running: "+str(datetime.datetime.now()))


In [2]:
df_id_zip=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/Email_Zips/output_loyalty_member_by_id.csv",dtype=str)
del df_id_zip['sign_up_date']
df_id_zip['customer_zip_code']=df_id_zip['customer_zip_code'].fillna("00000")
df_id_zip['customer_zip_code']=df_id_zip['customer_zip_code'].apply(lambda x: x.zfill(5)[0:5])
logging.info("Finished Reading Zip by member: "+str(datetime.datetime.now()))

In [8]:
df_id_zip.shape

(19249960, 2)

In [6]:
len(df_id_zip['customer_id_hashed'].unique())

19249960

In [59]:
data_1=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/MediaStormCustDtl.txt",header=None,dtype=str)
logging.info("Finished Reading Sales Data 1: "+str(datetime.datetime.now()))

data_2=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/MediaStorm customer transaction details - 2018-01-09 - 2018-03-31.txt",dtype=str)
logging.info("Finished Reading Sales Data 2: "+str(datetime.datetime.now()))

data_3=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/MediaStorm customer transaction details - 2018-04-01 - 2018-04-15.txt",dtype=str)
logging.info("Finished Reading Sales Data 3: "+str(datetime.datetime.now()))

data_1.columns=data_2.columns.tolist()  
data_1['customer_id_hashed']=data_1['customer_id_hashed'].apply(lambda x: hashlib.sha256(x.encode('UTF-8')).hexdigest())
logging.info("Finished Hashing: "+str(datetime.datetime.now()))
gc.collect()

def clean_data(df):
    del df['merch_cat']

    df['total_transaction_amt']=df['total_transaction_amt'].astype(float)
    df=df.drop_duplicates()    
    df['transaction_date']=df['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    df=df[['customer_id_hashed','transaction_date','total_transaction_amt']]
    df_sales=df.groupby(['customer_id_hashed','transaction_date'])['total_transaction_amt'].sum().to_frame().reset_index()
    df_trans=df[df['total_transaction_amt']>0]
    df_trans=df.groupby(['customer_id_hashed','transaction_date'])['total_transaction_amt'].count().to_frame().reset_index()
    df_trans=df_trans.rename(columns={"total_transaction_amt":"total_transaction_count"})
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','transaction_date'],how="left")
    return df
data_1=clean_data(data_1)
data_2=clean_data(data_2)
data_3=clean_data(data_3)
data_all=data_1.append(data_2).append(data_3).drop_duplicates()
data_all=data_all.drop_duplicates()

del data_1
del data_2
del data_3
gc.collect()

data_all=data_all[data_all['transaction_date']>=datetime.datetime(2017,5,7).date()]
data_all['weekday']=data_all['transaction_date'].apply(lambda x: x.weekday())

data_all['week_end_date']=np.where(data_all['weekday']==6,
                                   data_all['transaction_date'].apply(lambda x: x+datetime.timedelta(days=6)),
                                  data_all['transaction_date'].apply(lambda x: x+datetime.timedelta(days=5-x.weekday()))
                                  )
del data_all['transaction_date']
del data_all['weekday']
gc.collect()
logging.info("Finished Reading Sales by member: "+str(datetime.datetime.now()))


data_all=data_all.groupby(['customer_id_hashed','week_end_date'])['total_transaction_amt','total_transaction_count'].sum()

try:
    data_all=data_all.to_frame()
except:
    logging.info("No need to do 'to_frame' in data_all: "+str(datetime.datetime.now()))


data_all=data_all.reset_index()
    
try:
    del data_all['index']
except:
    logging.info("No index deleted: in data_all"+str(datetime.datetime.now()))
    
logging.info("Finished aggregating by zip by week: "+str(datetime.datetime.now()))
gc.collect()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [61]:
logging.info("Start Merging: "+str(datetime.datetime.now()))
data_all=pd.merge(data_all,df_id_zip,on="customer_id_hashed",how="left")
logging.info("Finished merging: "+str(datetime.datetime.now()))
data_all.to_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/sales_by_zip_20180529.csv",index=False)

(18726, 3)

In [71]:
gc.collect()
logging.info("Start Groupby on zip and week: "+str(datetime.datetime.now()))
data_all_46_full_weeks=data_all.groupby(['customer_zip_code','week_end_date'])['total_transaction_amt','total_transaction_count'].sum().reset_index()
gc.collect()
data_all_46_full_weeks.to_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/sales_by_zip_agg_20180529.csv",index=False)


In [12]:
# Do the last kernel above
data_all_46_full_weeks=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/sales_by_zip_20180529.csv",dtype=str)
data_all_46_full_weeks['total_transaction_count']=data_all_46_full_weeks['total_transaction_count'].astype(int)
data_all_46_full_weeks['total_transaction_amt']=data_all_46_full_weeks['total_transaction_amt'].astype(float)
data_all_46_full_weeks_agg=data_all_46_full_weeks.groupby(['customer_zip_code','week_end_date'])['total_transaction_amt','total_transaction_count'].sum().reset_index()


In [13]:
data_all_46_full_weeks_agg.head(2)

,customer_zip_code,week_end_date,total_transaction_amt,total_transaction_count
0,4421,2017-12-09,49.24,1
1,7754,2017-05-20,0.00,2


In [14]:
gc.collect()
data_all_46_full_weeks_agg.to_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/sales_by_zip_agg_20180529.csv",index=False)
